In [2]:
import numpy as np
from pedalboard import Pedalboard, Plugin, Chorus, Mix, Chain, Gain
from plgUtil import *
from paramOptim import optimize_plg_params
from faustify import faustify
import typing
from random import choice, randint, random, shuffle
from genAlgo import *
import time as ti

In [3]:
a = Pedalboard([Gain()])

In [4]:
a = Pedalboard([Gain()])
for _ in range(6) :
    mutate(a)

print(faustify(a))



Board after mutation: 
<Pedalboard with 2 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55c702b1e9b0>] at 0x55c702a265c0>, <pedalboard.Gain gain_db=1 at 0x55c702a3a720>]>
Board after mutation: 
<Pedalboard with 2 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55c702b1e9b0>] at 0x55c702a265c0>, <pedalboard.Mix with 2 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x55c702a3a720>] at 0x55c7029c08b0>, <pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x55c702ab4e80>] at 0x55c7029d65f0>] at 0x55c7029ad460>]>
Board after mutation: 
<Pedalboard with 2 plugins: [<pedalboard.Mix with 2 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55c702b1e9b0>] at 0x55c702a265c0>, <pedalboard.Chain with 1 plugin: [<pedalboard.Chorus r

In [5]:
def evolve(dry, wet, sr) :

    tol=1.0e-5

    N_pop = 12
    N_gens = 20
    N_survive = 10

    # models all held here
    models = [Pedalboard([])]

    # create initial population
    for plg in PLUGINS : 
        models += Pedalboard([copyPlg(plg)])

    models = generation(models, N_pop, N_survive)

    gen_num = 0
    converge = False
    elapsed = ti.time()


    while gen_num < N_gens :

        errors = [calc_error(model(dry, sr), wet) for model in models]
        aridxs = np.argsort(errors)[:N_pop]
        errors = [errors[i] for i in aridxs]
        models = [models[i] for i in aridxs]

        if errors[0] <= tol :
            converge = True
            break
            
        if N_pop < 48:
            N_pop += 4

        models = generation(models, N_pop, N_survive)
    
    conv = "converged" if converge else "did not converge"
    print(f"The model {conv}!")
    print(f"Best Model: {models[0]} \nError: {models[0]}")

    return models[0]

In [36]:
models = [Pedalboard([])]

# create initial population
for plg in PLUGINS : 
    print(f"PEDALBOARD ADD: {Pedalboard([copyPlg(plg)])}")
    models += [Pedalboard([copyPlg(plg)])]
print(len(models)) 
N_survive = 12
parent1 = rd.choice(models[:N_survive])
parent2 = rd.choice(models[:N_survive])
models += child(parent1, parent2)
print(len(models))

PEDALBOARD ADD: <Pedalboard with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x55c702d3cff0>]>
PEDALBOARD ADD: <Pedalboard with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x55c702b592d0>]>
PEDALBOARD ADD: <Pedalboard with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x55c702cf7eb0>]>
4
Parent1: <Pedalboard with 0 plugins: []>   ||  Parent2: <Pedalboard with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x55c702d3cff0>]>
Parents: [<Pedalboard with 0 plugins: []>, <Pedalboard with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x55c702d3cff0>]>]
Concatenating parents in srs/prl
5


In [20]:
N_survive = 12
parent1 = rd.choice(models[:N_survive])
parent2 = rd.choice(models[:N_survive])
parents = [parent1, parent2]
rd.shuffle(parents)
parents

[<pedalboard.Gain gain_db=1 at 0x5649a0cca0d0>,
 <pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x5649a0beb580>]

In [4]:
def generation(models, N, N_survive) :
    """
    oc: @jatinchowdhury18    
    create a generation of models from first two in existing list
    """
    while len(models) < N:
        parent1 = choice(models[:N_survive])
        parent2 = choice(models[:N_survive])
        models.append(child(parent1, parent2))

        # check for duplicates
        for m in models[:-1]:
            if m == models[-1]:
                # print('Duplicate detected! Mutating again...')
                models.pop()
                break

    return models

In [ ]:
def addSeries(board1, board2) :
    return Pedalboard(list(board1) + list(board2))

def addParallel(board1, board2) :
    return Pedalboard([Mix([Pedalboard(list(board1)), Pedalboard(list(board2))])])

def child(parent1, parent2) :
    is_mutation = (random() <= P_MUTATION)
    parents = shuffle([parent1, parent2])
    if is_mutation : 
        return add_mutation(parents[0])
    else : 
        return choice([addSeries, addParallel])(parents[0], parents[1])
        

0.5870069667555846

In [ ]:
def board_fitness(board, plg, input_buf, target_buf, samplerate, tol=1.0e-5) :
    return

In [84]:
board = Pedalboard([Pedalboard([])])
plg = list(board[0])
board.append(Chain(plg))
board


<Pedalboard with 2 plugins: [<pedalboard.Chain with 0 plugins: [] at 0x56210c118b80>, <pedalboard.Chain with 0 plugins: [] at 0x56210be0c9b0>]>

### Points to consider
- can insert

In [47]:
b = dir(list)
b.reverse()
b
help(list.insert)

Help on method_descriptor:

insert(self, index, object, /)
    Insert object before index.



In [45]:
a = [1]
list.extend(a, [2,3])
a

[1, 2, 3]

In [ ]:
from pedalboard import Pedalboard, Plugin, Chain, Mix, Gain, Chorus, Reverb
import time as ti
import numpy as np
import random as rd
from plgUtil import *
from paramOptim import *

P_MUTATION = 0.6


def evolve(dry, wet, sr) :

    tol=1.0e-5

    N_pop = 12
    N_gens = 20
    N_survive = 10

    # initialize models
    models = [Pedalboard([Chain([newPlg(plg)])]) for plg in PLUGINS]
    
    # do I want this here?
    models = generation(models, N_pop, N_survive)

    gen_num = 0
    converge = False
    elapsed = ti.time()


    while gen_num < N_gens :

        models = [m for m in models if m != None]
        errors = [calc_error(m(dry, sr), wet) for m in models]
        aridxs = np.argsort(errors)[:N_pop]
        errors = [errors[i] for i in aridxs]
        models = [models[i] for i in aridxs]

        if errors[0] <= tol :
            converge = True
            break
            
        if N_pop < 48:
            N_pop += 4

        models = generation(models, N_pop, N_survive)
    
    conv = "converged" if converge else "did not converge"
    print(f"The model {conv}!")
    print(f"Best Model: {models[0]} \nError: {models[0]}")

    return models[0]




def generation(models, N, N_survive):
    """ 
    create a generation of models from first two in existing list
    """
    while len(models) < N:

        for model in models :
            if model == None : models.remove(model)

        parents = [newBoard(rd.choice(models[:N_survive])) for _ in range(2)]
        # print(f"PARENT IN MODEL: {parents[0] in models or parents[1] in models}")
        # print(f"MODELS (type={type(models)}): {models}\n")
        # print(f"    PARENTS: {parents}\n")
        # print(f"    CHILD (type={type(child(parents))}): {child(parents)}\n")
        models += [child(parents)]

        # check for duplicates
        for model in models[:-1]:
            if model == models[:-1] :
                # print('Duplicate detected! Mutating again...')
                models.pop()
                break

    return models


def child(parents) :
    if rd.random() <= P_MUTATION : 
        return mutate(parents[0])

    else : 
        print(f"CONCAT PARENTS in srs/prl")
        return rd.choice([addSeries, addParallel])(parents[0], parents[1])


def mutate(board : Pedalboard) :
    """
    Add a plugin at a rd.random point in a Pedalboard
    """
    def chain_opt(elem) :
        return elem if isinstance(elem, Chain) else Chain([elem])

    # plugin in chain to mix
    def addAsMix(board, id, plg) :
        board[id] = Mix([chain_opt(board[id]), chain_opt(plg)])

    if board == Pedalboard([]) :
        return Pedalboard([rd.choice(PLUGINS)])
    
    r_id = rd.randint(0, len(board)-1) if len(board) > 1 else 0
    print(f"Mutation: r_id={r_id}\n     Board = {board}")
    curr = board[r_id]

    if isinstance(curr, Mix) : 
        mutate(rd.choice(curr))
    else : 
        r_plg = newPlg(rd.choice(PLUGINS))
        rd.choice([Pedalboard.insert, addAsMix])(board, r_id, Chain([r_plg]))
        print(f"Board after mutation: \n{board}")
        return Pedalboard(list(board))


